[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/emasquil/ppo/blob/main/ppo.ipynb)


# Proximal Policy Optimization (PPO) playground

Notebook for running PPO on simple environments from OpenAI Gym

## Install dependencies (only on Google Colab)

In [ ]:
# Installing our own implementation
! git clone https://github.com/emasquil/ppo.git
! pip install -e /content/ppo

# Visualization stuff
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev

## Imports

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Load autoreload extension
%load_ext autoreload
%autoreload 2

In [ ]:
import base64
import imageio
import IPython
import tqdm.notebook as tq
import numpy as np
import os

from acme import specs
from trax.jaxboard import SummaryWriter
import pyvirtualdisplay

# Set up a virtual display for rendering.
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

from ppo.agents import VanillaPPO, general_advantage_estimation
from ppo.env_wrapper import PendulumEnv
from ppo.networks import PolicyNetwork, ValueNetwork
from ppo.replay_buffers import DataLoader

### Visualization functions

In [ ]:
def display_video(frames, filename="temp.mp4", frame_repeat=1):
    """Save and display video."""
    # Write video
    with imageio.get_writer(filename, fps=60) as video:
        for frame in frames:
            for _ in range(frame_repeat):
                video.append_data(frame)
    # Read video and display the video
    video = open(filename, "rb").read()
    b64_video = base64.b64encode(video)
    video_tag = ('<video  width="320" height="240" controls alt="test" ' 'src="data:video/mp4;base64,{0}">').format(
        b64_video.decode()
    )
    return IPython.display.HTML(video_tag)

## Definitions

Definition of all the parts used in the learning loop: environment, agent, etc.

In [ ]:
environment = PendulumEnv()
environment_spec = specs.make_environment_spec(environment)

In [ ]:
# Training
num_training_iterations = 5
max_rollout = 300
gae_lambda =  0.95
num_epochs = 5
batch_size = 32
learning_rate = 1e-3
clipping_ratio_threshold = 0.2 
max_grad_norm = 0.5
discount = 0.99 

# Logs
log_dir="experiments"
experiment_name="pendulum_0"

# Keys
key_networks = 1
key_sampling_policy = 1
key_shuffling_batch = 1

In [ ]:
# Create the agent
def policy_network(observations):
    return PolicyNetwork((10, 20), environment_spec.actions, "policy")(observations)


def value_network(observations):
    return ValueNetwork((10, 20), "value")(observations)


agent = VanillaPPO(
    observation_spec=environment_spec.observations, 
    policy_network=policy_network, 
    value_network=value_network, 
    key_networks=key_networks,
    key_sampling_policy=key_sampling_policy,  
    learning_rate=learning_rate, 
    discount=discount, 
    clipping_ratio_threshold=clipping_ratio_threshold, 
    max_grad_norm=max_grad_norm
)

## Interaction loop

In [ ]:
def training_loop(
    environment,
    agent,
    num_training_iterations,
    num_epochs,
    max_rollout,
    batch_size,
    log_dir,
    experiment_name,
):
    """
    Main training loop
    """
    # Initializing counters
    all_returns = []
    avg_reward = 0.0
    writer = SummaryWriter(os.path.join(log_dir, experiment_name))
    # Counter to keep track of the global timestep
    t = 0

    pbar = tq.tqdm(range(num_training_iterations), position=0)
    pbar.set_description("Training loop iteration")
    for iteration in pbar:
        # Annealing

        # Rollout phase
        # Reset any counts and start the environment.
        agent.replay_buffer.clear()
        timestep = environment.reset()
        rollout_return = 0

        # Make the first observation.
        agent.observe_first(timestep)

        pbar_rollout = tq.tqdm(range(max_rollout), position=1, leave=False)
        pbar_rollout.set_description(f"Rollout step")
        for rollout_step in pbar_rollout:
            pbar_rollout.set_postfix(reward=timestep.reward)
            value = agent.get_value(timestep.observation)
            action, log_prob = agent.select_action_and_prob(timestep.observation)
            timestep = environment.step(action)
            agent.observe(value, log_prob, action, timestep)
            rollout_return += timestep.reward
            # Incrementing count of time step
            t += 1
            # Avg reward per global timestep
            avg_reward += (timestep.reward - avg_reward) / t
            # Add avg reward
            writer.scalar("return/avg_per_timestep", avg_reward, t)
            if timestep.last():
                break

        # Learning phase
        dataloader = DataLoader(agent.replay_buffer, batch_size, key_shuffling_batch)
        trajectory = dataloader.get_full_memory()
        advantages = general_advantage_estimation(trajectory, agent, timestep, discount, gae_lambda)
        agent.add_advantage(advantages)

        # Add cummulative return of the episode to tensorboard
        writer.scalar("return/cummulative_per_rollout", rollout_return, iteration)

        pbar_epochs = tq.tqdm(range(num_epochs), leave=False, position=1)
        pbar_epochs.set_description(f"Epoch")
        for epoch in pbar_epochs:
            dataloader.shuffle()
            value_losses = []
            policy_losses = []
            for batch in dataloader:
                value_loss, policy_loss = agent.update(batch)
                value_losses.append(value_loss)
                policy_losses.append(policy_loss)
            writer.scalar("loss/value", np.mean(value_losses), epoch + iteration * num_epochs)
            writer.scalar("loss/policy", np.mean(policy_losses), epoch + iteration * num_epochs)

        all_returns.append(rollout_return)
    return all_returns

In [ ]:
def evaluate(environment, agent, evaluation_episodes):
    frames = []

    pbar = tq.tqdm(range(evaluation_episodes))
    pbar.set_description("Episode")
    for episode in pbar:
        timestep = environment.reset()
        episode_return = 0
        steps = 0
        while not timestep.last():
            frames.append(environment.render(mode="rgb_array"))
            action = agent.select_action(timestep.observation)
            timestep = environment.step(action)
            steps += 1
            episode_return += timestep.reward
        print(f"Episode {episode} ended with reward {episode_return} in {steps} steps")
    return frames

In [ ]:
%tensorboard --logdir experiments

### Train

In [ ]:
training_loop(agent=agent, environment=environment, max_rollout=max_rollout, batch_size=batch_size, num_training_iterations=num_training_iterations, num_epochs=num_epochs, log_dir=log_dir, experiment_name=experiment_name)

### Evaluate

In [ ]:
display_video(evaluate(agent=agent, environment=environment, evaluation_episodes=5))